# RDFIA: Practical work 1-cd
## Convolutional Neural Networks
### Sorbonne Université 2024
### [Course link](https://rdfia.github.io/)
### GRUSS Carlos, RCHAKI Oussama


In [ ]:
#!git clone https://github.com/cdancette/deep-learning-polytech-tp6-7.git

# Only run if "utils.py" is not already downloaded
if not os.path.exists("utils.py"):
    # Download the file
    ! wget https://github.com/rdfia/rdfia.github.io/raw/master/code/2-cd/utils.py

%run "utils.py"

In [ ]:
import argparse
import os
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torchvision.transforms import v2
import torchvision.datasets as datasets
import torch.optim.lr_scheduler as lr_scheduler
from sklearn.decomposition import PCA
from torchvision.transforms import Lambda
from sklearn.preprocessing import MinMaxScaler

from utils import *

PRINT_INTERVAL = 200
PATH="datasets"

In [4]:
def pca_whitening(image):
    """
    Apply PCA whitening to an image. Assumes the input is a 3D numpy array (H x W x C).
    """
    flat_img = image.reshape(-1, 3)
    # Apply PCA
    pca = PCA(whiten=True)
    whitened = pca.fit_transform(flat_img)
    # Reshape to original dimensions
    whitened_img = whitened.reshape(image.shape)
    return torch.from_numpy(whitened_img).float()

class ConvNet(nn.Module):
    """
    This class defines the structure of the neural network
    """
    def __init__(self):
        super(ConvNet, self).__init__()
        # We first define the convolution and pooling layers as a features extractor
        self.features = nn.Sequential(
            nn.Conv2d(1, 6, (5, 5), stride=1, padding=2),
            nn.Tanh(),
            nn.MaxPool2d((2, 2), stride=2, padding=0),
            nn.Conv2d(6, 16, (5, 5), stride=1, padding=0),
            nn.Tanh(),
            nn.MaxPool2d((2, 2), stride=2, padding=0),
        )
        # We then define fully connected layers as a classifier
        self.classifier = nn.Sequential(
            nn.Linear(400, 120),
            nn.Tanh(),
            nn.Linear(120, 84),
            nn.Tanh(),
            nn.Linear(84, 10)
            # Reminder: The softmax is included in the loss, do not put it here
        )

    # Method called when we apply the network to an input batch
    def forward(self, input):
        bsize = input.size(0) # batch size
        output = self.features(input) # output of the conv layers
        output = output.view(bsize, -1) # we flatten the 2D feature maps into one 1D vector for each input
        output = self.classifier(output) # we compute the output of the fc layers
        return output

class ConvNet_v2(nn.Module):
    """
    This class defines the structure of the neural network
    """
    def __init__(self):
        super(ConvNet_v2, self).__init__()
        # We first define the convolution and pooling layers as a features extractor
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(5, 5), stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2, padding=0),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(5, 5), stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2, padding=0),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(5, 5), stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2, padding=0, ceil_mode=True)
        )
        # We then define fully connected layers as a classifier
        self.classifier = nn.Sequential(
            nn.Linear(in_features=1024, out_features=1000),
            nn.ReLU(),
            nn.Linear(1000, 10)
            # Reminder: The softmax is included in the loss, do not put it here
        )

    # Method called when we apply the network to an input batch
    def forward(self, input):
        bsize = input.size(0) # batch size
        output = self.features(input) # output of the conv layers
        output = output.view(bsize, -1) # we flatten the 2D feature maps into one 1D vector for each input
        output = self.classifier(output) # we compute the output of the fc layers
        return output

class ConvNet_v3(nn.Module):
    """
    This class defines the structure of the neural network
    """
    def __init__(self, prob=0.5):
        super(ConvNet_v3, self).__init__()
        # We first define the convolution and pooling layers as a features extractor
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(5, 5), stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2, padding=0),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(5, 5), stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2, padding=0),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(5, 5), stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2, padding=0, ceil_mode=True)
        )
        # We then define fully connected layers as a classifier
        self.classifier = nn.Sequential(
            nn.Linear(in_features=1024, out_features=1000),
            nn.ReLU(),
            nn.Dropout(p=prob),
            nn.Linear(1000, 10)
            # Reminder: The softmax is included in the loss, do not put it here
        )

    # Method called when we apply the network to an input batch
    def forward(self, input):
        bsize = input.size(0) # batch size
        output = self.features(input) # output of the conv layers
        output = output.view(bsize, -1) # we flatten the 2D feature maps into one 1D vector for each input
        output = self.classifier(output) # we compute the output of the fc layers
        return output
    
class ConvNet_v4(nn.Module):
    """
    This class defines the structure of the neural network
    """
    def __init__(self):
        super(ConvNet_v4, self).__init__()
        # We first define the convolution and pooling layers as a features extractor
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(5, 5), stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2, padding=0),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(5, 5), stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2, padding=0),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(5, 5), stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2, padding=0, ceil_mode=True)
        )
        # We then define fully connected layers as a classifier
        self.classifier = nn.Sequential(
            nn.Linear(in_features=1024, out_features=1000),
            nn.ReLU(),
            nn.Linear(1000, 10)
            # Reminder: The softmax is included in the loss, do not put it here
        )

    # Method called when we apply the network to an input batch
    def forward(self, input):
        bsize = input.size(0) # batch size
        output = self.features(input) # output of the conv layers
        output = output.view(bsize, -1) # we flatten the 2D feature maps into one 1D vector for each input
        output = self.classifier(output) # we compute the output of the fc layers
        return output

class ConvNet_v5(nn.Module):
    """
    This class defines the structure of the neural network
    """
    def __init__(self, prob=0.5):
        super(ConvNet_v5, self).__init__()
        # We first define the convolution and pooling layers as a features extractor
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(5, 5), stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2, padding=0),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(5, 5), stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2, padding=0),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(5, 5), stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2, padding=0, ceil_mode=True)
        )
        # We then define fully connected layers as a classifier
        self.classifier = nn.Sequential(
            nn.Linear(in_features=1024, out_features=1000),
            nn.ReLU(),
            nn.Dropout(p=prob),
            nn.Linear(1000, 10)
            # Reminder: The softmax is included in the loss, do not put it here
        )

    # Method called when we apply the network to an input batch
    def forward(self, input):
        bsize = input.size(0) # batch size
        output = self.features(input) # output of the conv layers
        output = output.view(bsize, -1) # we flatten the 2D feature maps into one 1D vector for each input
        output = self.classifier(output) # we compute the output of the fc layers
        return output

def get_dataset(batch_size, cuda=False, task="mnist", transform=None, augment=None):
    """
    This function loads the dataset and performs transformations on each
    image (listed in `transform = ...`).
    """
    if task == "mnist": 
        train_dataset = datasets.MNIST(PATH, train=True, download=True, 
                                    transform=transforms.Compose([transforms.ToTensor()]))
        val_dataset = datasets.MNIST(PATH, train=False, download=True, 
                                    transform=transforms.Compose([transforms.ToTensor()]))
    elif task == "cifar10":
        transform_train = [transforms.ToTensor()]
        transform_val = [transforms.ToTensor()]

        # Perform data augmentation if specified
        if augment == "standard":
            transform_train.extend([transforms.RandomCrop(28), transforms.RandomHorizontalFlip()])
            transform_val.extend([transforms.CenterCrop(28)])
        elif augment == "cutmix":
            transform_train.append(v2.CutMix())
            transform_val.append(v2.CutMix())

        # Add the normalization transform if specified
        if transform == "normalize":
            mean = [0.491, 0.482, 0.447]
            std = [0.202, 0.199, 0.201]
            transform_train.append(transforms.Normalize(mean, std))
            transform_val.append(transforms.Normalize(mean, std))
        elif transform == "pca":
            transform_train.append(Lambda(pca_whitening))
            transform_val.append(Lambda(pca_whitening))
    
        train_dataset = datasets.CIFAR10(PATH, train=True, download=True, 
                                    transform=transforms.Compose(transform_train))
        val_dataset = datasets.CIFAR10(PATH, train=False, download=True, 
                                transform=transforms.Compose(transform_val))
        
        # Special processing is required if we want to use MinMaxScaler
        if transform == "minmax":
            # Convert the dataset to NumPy arrays and reshape to 2D for MinMaxScaler
            x_train = np.array([data[0].numpy() for data in train_dataset])
            x_train = x_train.reshape(x_train.shape[0], -1) # Reshape to 2D
            x_val = np.array([data[0].numpy() for data in val_dataset])
            x_val = x_val.reshape(x_val.shape[0], -1) # Reshape to 2D

            scaler = MinMaxScaler()
            
            # Apply MinMaxScaler on the reshaped data
            train_normalized = scaler.fit_transform(x_train)
            val_normalized = scaler.fit_transform(x_val)

            # Reshape back to original image shape and convert to tensors
            train_normalized = train_normalized.reshape(x_train.shape[0], 3, 32, 32)
            train_normalized = torch.from_numpy(train_normalized).float()
            val_normalized = val_normalized.reshape(x_val.shape[0], 3, 32, 32)
            val_normalized = torch.from_numpy(val_normalized).float()

            # Combine normalized data with labels
            train_dataset = list(zip(train_normalized, [data[1] for data in train_dataset]))
            val_dataset = list(zip(val_normalized, [data[1] for data in val_dataset]))

    train_loader = torch.utils.data.DataLoader(train_dataset,
                        batch_size=batch_size, shuffle=True, pin_memory=cuda, num_workers=2)
    val_loader = torch.utils.data.DataLoader(val_dataset,
                        batch_size=batch_size, shuffle=False, pin_memory=cuda, num_workers=2)

    return train_loader, val_loader

def epoch(data, model, criterion, optimizer=None, cuda=False, verbose=True):
    """
    Make a pass (called epoch in English) on the data `data` with the
     model `model`. Evaluates `criterion` as loss.
     If `optimizer` is given, perform a training epoch using
     the given optimizer, otherwise, perform an evaluation epoch (no backward)
     of the model.
    """

    # indicates whether the model is in eval or train mode (some layers behave differently in train and eval)
    model.eval() if optimizer is None else model.train()

    # objects to store metric averages
    avg_loss = AverageMeter()
    avg_top1_acc = AverageMeter()
    avg_top5_acc = AverageMeter()
    avg_batch_time = AverageMeter()
    global loss_plot

    # we iterate on the batches
    tic = time.time()
    for i, (input, target) in enumerate(data):

        if cuda: # only with GPU, and not with CPU
            input = input.cuda()
            target = target.cuda()

        # forward
        output = model(input)
        loss = criterion(output, target)

        # backward if we are training
        if optimizer:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # compute metrics
        prec1, prec5 = accuracy(output, target, topk=(1, 5))
        batch_time = time.time() - tic
        tic = time.time()

        # update
        avg_loss.update(loss.item())
        avg_top1_acc.update(prec1.item())
        avg_top5_acc.update(prec5.item())
        avg_batch_time.update(batch_time)
        # if optimizer and plot_batch_loss:
        #     loss_plot.update(avg_loss.val)
        # print info
        if i % PRINT_INTERVAL == 0:
            if verbose:
                print('[{0:s} Batch {1:03d}/{2:03d}]\t'
                    'Time {batch_time.val:.3f}s ({batch_time.avg:.3f}s)\t'
                    'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                    'Prec@1 {top1.val:5.1f} ({top1.avg:5.1f})\t'
                    'Prec@5 {top5.val:5.1f} ({top5.avg:5.1f})'.format(
                    "EVAL" if optimizer is None else "TRAIN", i, len(data), batch_time=avg_batch_time, loss=avg_loss,
                    top1=avg_top1_acc, top5=avg_top5_acc))
            # if optimizer and plot_batch_loss:
            #     loss_plot.plot()

    # Print summary
    if verbose:
        print('\n===============> Total time {batch_time:d}s\t'
            'Avg loss {loss.avg:.4f}\t'
            'Avg Prec@1 {top1.avg:5.2f} %\t'
            'Avg Prec@5 {top5.avg:5.2f} %\n'.format(
            batch_time=int(avg_batch_time.sum), loss=avg_loss,
            top1=avg_top1_acc, top5=avg_top5_acc))

    return avg_top1_acc, avg_top5_acc, avg_loss

def plot_metrics(loss_histories, acc_histories, epochs, plot='same'):
    if plot == 'separate':
        # Create a 2x2 subplot grid
        fig, axs = plt.subplots(2, 2, figsize=(20, 10))

        epochs_range = range(1, epochs + 1)

        # Plot Train Accuracy
        ax = axs[0, 0]
        for params, metrics in acc_histories.items():
            ax.plot(epochs_range, metrics['train'], label=params)
        ax.set_title('Train Accuracy')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Accuracy (%)')
        ax.legend()
        ax.grid(False)

        # Plot Test Accuracy
        ax = axs[0, 1]
        for params, metrics in acc_histories.items():
            ax.plot(epochs_range, metrics['val'], label=params)
        ax.set_title('Test Accuracy')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Accuracy (%)')
        ax.legend()
        ax.grid(False)

        # Plot Train Loss
        ax = axs[1, 0]
        for params, metrics in loss_histories.items():
            ax.plot(epochs_range, metrics['train'], label=params)
        ax.set_title('Train Loss')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Loss')
        ax.legend()
        ax.grid(False)

        # Plot Test Loss
        ax = axs[1, 1]
        for params, metrics in loss_histories.items():
            ax.plot(epochs_range, metrics['val'], label=params)
        ax.set_title('Test Loss')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Loss')
        ax.legend()
        ax.grid(False)
    elif plot == 'same':
        # Create a 1x2 subplot grid
        # Plot train and test accuracy on one plot
        # and train and test loss on the other
        fig, axs = plt.subplots(1, 2, figsize=(20, 10))

        epochs_range = range(1, epochs + 1)

        # Plot Accuracy
        ax = axs[0]
        for params, metrics in acc_histories.items():
            ax.plot(epochs_range, metrics['train'], label=f"{params} (train)")
            ax.plot(epochs_range, metrics['val'], label=f"{params} (test)")
        ax.set_title('Accuracy')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Accuracy (%)')
        ax.legend()
        ax.grid(False)

        # Plot Loss
        ax = axs[1]
        for params, metrics in loss_histories.items():
            ax.plot(epochs_range, metrics['train'], label=f"{params} (train)")
            ax.plot(epochs_range, metrics['val'], label=f"{params} (test)")
        ax.set_title('Loss')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Loss')
        ax.legend()
        ax.grid(False)
    else:
        raise ValueError("plot should be in ['same', 'separate']")

    # Adjust layout and display the plot
    plt.tight_layout()
    plt.show()

def main(bs_list=[128], 
         lr_list=[0.1], 
         epochs=5, 
         cuda=False, 
         task="mnist", 
         verbose=True, 
         transform=None, 
         plot='same', 
         augment=None,
         scheduler=False,
         model='ConvNet',
         prob=0.5,
         optimizer="SGD"):
    
    # Initialize dictionaries to store loss and accuracy histories
    loss_histories = {}
    acc_histories = {}
    
    for batch_size in bs_list:
        # Get the data once outside the loop
        train_loader, test_loader = get_dataset(batch_size, cuda, task, transform=transform, augment=augment)
        for lr in lr_list:
            print(f"\nTraining with lr={lr} and batch_size={batch_size}\n")
            
            # Define model, loss, optimizer for each learning rate
            if model == "ConvNet":
                model = ConvNet()
            elif model == "ConvNet_v2":
                model = ConvNet_v2()
            elif model == "ConvNet_v3":
                model = ConvNet_v3(prob=prob)
            elif model == "ConvNet_v4":
                model = ConvNet_v4()
            elif model == "ConvNet_v5":
                model = ConvNet_v5(prob=prob)
            else:
                raise ValueError("model should be in ['ConvNet', 'ConvNet_v2', 'ConvNet_v3', 'ConvNet_v4', 'ConvNet_v5']")
            
            criterion = nn.CrossEntropyLoss()
            if optimizer == "Adam":
                optimizer = torch.optim.Adam(model.parameters(), lr)
            elif optimizer == "SGD":
                optimizer = torch.optim.SGD(model.parameters(), lr)
            else:
                raise ValueError("optimizer should be in ['Adam', 'SGD']")

            if scheduler:
                lr_sched = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

            if cuda:
                cudnn.benchmark = True
                model = model.cuda()
                criterion = criterion.cuda()
            
            # Initialize lists to store metrics per epoch
            train_losses = []
            val_losses = []
            train_accuracies = []
            val_accuracies = []
            
            # We iterate over epochs
            for epoch_num in range(epochs):
                if verbose:
                    print(f"=== Epoch {epoch_num+1}/{epochs} ===")
                
                # Train phase
                top1_acc, top5_acc, train_loss = epoch(train_loader, model, criterion, optimizer, cuda, verbose)
                train_losses.append(train_loss.avg)
                train_accuracies.append(top1_acc.avg)
                
                # Validation phase
                val_top1_acc, val_top5_acc, val_loss = epoch(test_loader, model, criterion, cuda=cuda, verbose=verbose)
                val_losses.append(val_loss.avg)
                val_accuracies.append(val_top1_acc.avg)

                if scheduler:
                    lr_sched.step()
            
            # Store the metrics history for these parameters
            params = f"lr={lr} bs={batch_size}"
            loss_histories[params] = {'train': train_losses, 'val': val_losses}
            acc_histories[params] = {'train': train_accuracies, 'val': val_accuracies}
    
    # After all learning rates have been processed, plot all metrics
    plot_metrics(loss_histories, acc_histories, epochs, plot)

### Instructions

- **bs_list** (`list`, default `[128]`): A list of batch sizes to use during training. The function will iterate over each batch size in the list.

- **lr_list** (`list`, default `[0.1]`): A list of learning rates to use for the optimizer. The function will train the model separately for each learning rate.

- **epochs** (`int`, default `5`): The number of epochs to train the model for each combination of batch size and learning rate.

- **cuda** (`bool`, default `False`): Whether to use CUDA (GPU acceleration) for training. Set to `True` if you have a compatible GPU and CUDA installed.

- **task** (`str`, default `"mnist"`): The dataset to use. Options are:
  - `"mnist"`: Use the MNIST dataset.
  - `"cifar10"`: Use the CIFAR-10 dataset.

- **verbose** (`bool`, default `True`): Whether to print detailed training progress and metrics.

- **transform** (`str` or `None`, default `None`): Data transformations to apply. Options are:
  - `"normalize"`: Apply normalization to the dataset.
  - `"pca"`
  - `"minmax"`
  - `None`: No additional transformations.

- **plot** (`str`, default `'same'`): How to plot the training metrics after training. Options are:
  - `'same'`: Plot training and validation accuracy and loss on the same graphs.
  - `'separate'`: Plot training and validation metrics on separate graphs.

- **augment** (`str` or `None`, default `None`): Whether to apply data augmentation techniques to the training data (only applicable when `task="cifar10"`).
  - `"standard"`: Apply the basic augmentation technique discussed in the report (crops and random horizontal flips).
  - `"cutmix"`: Apply CutMix transform to the dataset.

- **scheduler** (`bool`, default `False`): Whether to use a learning rate scheduler during training.

- **model** (`str`, default `'ConvNet'`): The model architecture to use. Options are:
  - `'ConvNet'`: The base convolutional neural network.
  - `'ConvNet_v2'`: A variant with increased capacity.
  - `'ConvNet_v3'`: Adds dropout layers.
  - `'ConvNet_v4'`: Adds batch normalization layers.
  - `'ConvNet_v5'`: Combines batch normalization and dropout.

- **prob** (`float`, default `0.5`): Dropout probability. Only used if the selected model includes dropout layers (e.g., `'ConvNet_v3'` or `'ConvNet_v5'`).

-**optimizer** (`str`, default `'SGD'`): Selects the optimizer.
  - `'SGD'`: Stochastic Gradient Descent.
  - `'Adam'`

In [ ]:
lrs = [0.1]
bs_list = [128]
main(bs_list=bs_list, lr_list=lrs, cuda=True, epochs=60, task="cifar10", 
     model="ConvNet_v2", plot='same', transform='pca', augment=None, scheduler=False, optimizer="SGD")